In [1]:
#include <memory>
#include <iostream>
#include <map>
#include <string>

In [2]:
class Resource
{
public:
    int number;
    
	Resource(int n) {
        number = n;
        std::cout << "Resource acquired\n"; 
    }
    
	~Resource() { 
        std::cout << "Resource destroyed\n"; 
    }
    
    friend std::ostream& operator<<(std::ostream& out, const Resource& res)
	{
		out << "I am a resource\n";
		return out;
	}
};

# 1 basic operation

## 1.a How to construct a std::unique_ptr

method1: use new

In [3]:
std::unique_ptr<Resource> res1{new Resource(5)}

Resource acquired


method2: use std:make_unique

In [4]:
std::unique_ptr<Resource> res2 = std::make_unique<Resource>(5);

Resource acquired


In [5]:
{
    std::unique_ptr<Resource> res3 = std::make_unique<Resource>(5);
    
    //dereference the pointer
    std::cout << *res3 << std::endl;
}

Resource acquired
I am a resource

Resource destroyed


## 1.b how std::unique_ptr work and the lifetime of smart_pointer and  the Resource it point to.

In [6]:
int do_something1(){
    
    std::unique_ptr<Resource> res1 =  std::make_unique<Resource>(2);
    
    std::cout << "exiting " << std::endl;
    return 0;
}; //res1 go out of scope, and then release Resource

In [7]:
int a = do_something1();

Resource acquired
exiting 
Resource destroyed


## 1.c  One resource, one std::unique_ptr: copy construct and copy assgnment disabled

In [8]:
int test_rule_of_five(){
    std::unique_ptr<Resource> p1{ new Resource{22} }; // Resource created here
	std::unique_ptr<Resource> p2{}; // Start as nullptr
 
	std::cout << "p1 is " << (static_cast<bool>(p1) ? "not null\n" : "null\n");
	std::cout << "p2 is " << (static_cast<bool>(p2) ? "not null\n" : "null\n");
 
    // //Copy construction is not allowed
    //unique_ptr<MyObject> p3(p1);
	
    // res2 = res1; // Won't compile: copy assignment is disabled
    
    // the output of std:move is a rvalue, the call the move assignment operator rather than copy assignment which is disabled 
	p2 = std::move(p1); // res2 assumes ownership, res1 is set to null
 
	std::cout << "Ownership transferred\n";
 
	std::cout << "p1 is " << (static_cast<bool>(p1) ? "not null\n" : "null\n");
	std::cout << "p2 is " << (static_cast<bool>(p2) ? "not null\n" : "null\n");
     
    std::cout << *p2 << std::endl;
    
	return 0;
};

In [9]:
int b = test_rule_of_five();

Resource acquired
p1 is not null
p2 is null
Ownership transferred
p1 is null
p2 is not null
I am a resource

Resource destroyed


## 1-d: Passing std::unique_ptr to a function

In [10]:
void takeOwnership(std::unique_ptr<Resource> res){
     if (res)
          std::cout << *res << '\n';
}; // the Resource is destroyed here


In [11]:
int test_unique_pointer_as_function_parameter1(){
    
    auto ptr{ std::make_unique<Resource>(111)};
 
//    takeOwnership(ptr); // This doesn't work, need to use move semantics
    takeOwnership(std::move(ptr)); // ok: use move semantics
 
    std::cout << "Ending program\n";
 
    return 0;
};

int d1 = test_unique_pointer_as_function_parameter1();

Resource acquired
I am a resource

Resource destroyed
Ending program


In [12]:
// The function only uses the resource, so we'll accept a pointer to the resource, not a reference to the whole std::unique_ptr<Resource>
void justUseResource(Resource* res){
    if (res)
          std::cout << *res << '\n';
};

In [13]:
int test_unique_pointer_as_function_parameter2(){
    
    auto ptr{ std::make_unique<Resource>(111)};
 
    justUseResource(ptr.get()); // ok
 
    std::cout << "Ending program\n";
 
    return 0;
};

int d2 = test_unique_pointer_as_function_parameter2();

Resource acquired
I am a resource

Ending program
Resource destroyed


## 1-e: Returning std::unique_ptr from a function

In [14]:
std::unique_ptr<Resource> createUniquePointer(){
  //create a local unique pointer
  std::unique_ptr<Resource> p1(new Resource(111));

  return p1; // p1 will surrender ownership
};


In [15]:
void test_unique_pointer_as_function_return(){
    std::unique_ptr<Resource> a1; //unique_ptr points to nothing
    a1=createUniquePointer(); //now a1 owns the object, here call the move assignment operator
    
    std::cout << *a1 << std::endl;
    std::cout << "program end " << std::endl;
};

test_unique_pointer_as_function_return();

Resource acquired
I am a resource

program end 
Resource destroyed


## 1-f: unique_pointer in dict

In [17]:
using ResourceMap = std::map<std::string, std::unique_ptr<Resource>>;

In [18]:
ResourceMap myMap;
myMap["000001"] = std::make_unique<Resource>(11);
myMap["000002"] = std::make_unique<Resource>(21);
myMap["000003"] = std::make_unique<Resource>(31);

Resource acquired
Resource acquired
Resource acquired


In [19]:
using RMI = std::map<std::string, std::unique_ptr<Resource>>::iterator;

In [20]:
RMI it = myMap.find("000002");

In [23]:
std::cout << it -> second -> number << std::endl;

21


In [25]:
std::cout << (it++) -> second -> number << std::endl;

31
